## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
import os

Using TensorFlow backend.


In [2]:
batch_size = 500 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

def cifar_generator(x,y, batch_size):
    while True:
        for indexs in range(0, len(x), batch_size):
            images = x[indexs: indexs+batch_size]
            labels = y[indexs: indexs+batch_size]
            augmented_iamges = next(data_generator.flow(images, labels, batch_size=batch_size, shuffle=False))
            yield augmented_iamges

train_gen = cifar_generator(x_train,y_train,batch_size)
test_gen = cifar_generator(x_test,y_test,batch_size)

In [5]:
# 建立 ResNet 模型
model = resnet(input_shape=(32,32,3)) 
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [6]:
history = model.fit_generator(train_gen,
                              steps_per_epoch=100,
                              epochs=epochs,
                              verbose=1)

Epoch 1/10
100/100 [==============================] - 2979s 30s/step - loss: 2.2240 - accuracy: 0.3998
Epoch 2/10
100/100 [==============================] - 2853s 29s/step - loss: 1.8167 - accuracy: 0.5237
Epoch 3/10
100/100 [==============================] - 2856s 29s/step - loss: 1.6182 - accuracy: 0.5831
Epoch 4/10
100/100 [==============================] - 2854s 29s/step - loss: 1.4860 - accuracy: 0.6214
Epoch 5/10
100/100 [==============================] - 2877s 29s/step - loss: 1.3841 - accuracy: 0.6511
Epoch 6/10
100/100 [==============================] - 2862s 29s/step - loss: 1.2990 - accuracy: 0.6748
Epoch 7/10
100/100 [==============================] - 2854s 29s/step - loss: 1.2280 - accuracy: 0.6963
Epoch 8/10
100/100 [==============================] - 2869s 29s/step - loss: 1.1551 - accuracy: 0.7175
Epoch 9/10
100/100 [==============================] - 2857s 29s/step - loss: 1.1084 - accuracy: 0.7319
Epoch 10/10
100/100 [==============================] - 2857s 29s/step - l

In [7]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.6467700231552125
Test accuracy: 0.6049000024795532


In [8]:
# Save model and weights
save_dir = os.path.join(os.getcwd(), 'Day100_saved_models')
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
init_model_file_name = 'Day100_model'
model_path = os.path.join(save_dir, init_model_file_name)
model.save(model_path)

In [ ]:
history = model.fit_generator(train_gen,
                              steps_per_epoch=100,
                              epochs=epochs,
                              verbose=1)

Epoch 1/10


In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])